Importar librerias

In [38]:
import requests
#Embeddings
from langchain_ollama import OllamaEmbeddings
#LLM
from langchain_ollama.llms import OllamaLLM
# Prompts
from langchain_core.prompts import ChatPromptTemplate
# doc loaders
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader, Docx2txtLoader
# ChromaDB
from langchain_community.vectorstores import Chroma
# text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
# runnable
from langchain_core.runnables import RunnablePassthrough
# Tools
from langchain.tools import tool
# LLM ollama
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

ImportError: cannot import name 'create_tool_calling_agent' from 'langchain.agents' (c:\Users\Alan Contreras P\Documents\1-escuela\Profe\8_semestre\Multidisciplinar\Codigos\1-Vertiche-BackEnd\langchainENV\.venv\Lib\site-packages\langchain\agents\__init__.py)

In [23]:
#????
#from langchain_community.document_loaders import WebBaseLoader
#from bs4 import BeautifulSoup

# Preparar documentos

## Cargar docs

In [24]:
# Cargar todos los PDFs de la carpeta "Contratos" con PyPDFLoader
loader = DirectoryLoader(
    path="./Contratos",
    glob="**/*.pdf",        # Cargar solo archivos PDF
    loader_cls=PyPDFLoader  # Cargar solo archivos PDF
)

documentos = loader.load()

## Chunking

In [25]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,       # caracteres por chunk
    chunk_overlap=50      # solapamiento entre chunks (mantiene contexto)
)

chunks = splitter.split_documents(documentos)

# Embeddings

In [26]:
embeddings = OllamaEmbeddings(model="llama3.2")

# Chroma

## Crear/Cargar base de datos vectorial

In [27]:
Crear = False
if Crear: #Crear la base de datos (solo la primera vez)
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory="./chroma_db"
    )
else:     # Cargar la base de datos ya creada
    vectorstore = Chroma(
        persist_directory="./chroma_db",
        embedding_function=embeddings
    )

### Agregar nuevos documentos

In [28]:
#vectorstore.add_documents(nuevos_chunks)

# RAG

In [29]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# LLM (borrar)

## Definir  tools

### Templates para prompts (no son finales)

## Tool Calling Agent 

## Entradas y salidas

Cómo fluye una petición real
```
Usuario: "busca contratos de oficina del 2023"
         ↓
LLM lee el input + descripción de tools
         ↓
Decide: filter_search(query="oficina", tipo="oficina", año=2023)
         ↓
LangChain ejecuta filter_search automáticamente
         ↓
El resultado (docs de Chroma) regresa al LLM
         ↓
LLM formula respuesta final con esa información
         ↓
Usuario recibe respuesta en lenguaje natural
```

## Pruebas

# LLM2

## Agente

###  LLM compartido 


In [30]:
llm = ChatOllama(model="llama3.2", temperature=0)

###  Chains internas + tools

In [31]:
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

#### Analysis

In [32]:
tem_chat_analysis = """ 
Eres un asistente especializado en contratos de arrendamiento. 
Tu función es responder preguntas, proporcionar resúmenes y realizar análisis generales sobre los contratos de arrendamiento.
Utiliza la información disponible en los documentos para ofrecer respuestas precisas y detalladas a las consultas de los usuarios.
"""

chain_analysis = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(tem_chat_analysis + """

Contexto de los documentos:
{context}

Pregunta o solicitud: {question}
""")
    | llm
)

@tool
def chat_analysis(query: str) -> str:
    """Analiza, resume o responde preguntas generales sobre contratos de arrendamiento."""
    result = chain_analysis.invoke(query)
    return result.content

#### Filter

In [33]:
tem_filter_search  = """
Eres un asistente especializado en buscar y filtrar contratos de arrendamiento.
Tu función es ayudar a los usuarios a encontrar contratos específicos basados en criterios como fecha, tipo de contrato, partes involucradas o ubicación.
"""

chain_filter = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(tem_filter_search + """

Contratos encontrados:
{context}

Búsqueda solicitada: {question}
""")
    | llm
)

@tool
def filter_search(query: str, tipo: str = None, año: int = None) -> str:
    """Busca contratos específicos por filtros como tipo, año o arrendador."""
    result = chain_filter.invoke(query)
    return result.content

#### Generation

In [34]:
tem_generation_tool = """
Eres un asistente especializado en crear o sugerir cláusulas, generar borradores o redactar términos de contratos de arrendamiento.
Tu función es ayudar a los usuarios a generar contenido relacionado con contratos de arrendamiento, ya sea creando nuevas cláusulas, sugiriendo redacciones o generando borradores completos.
"""

chain_generation = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(tem_generation_tool + """

Cláusulas de referencia encontradas:
{context}

Solicitud de generación: {question}

AVISO: Las sugerencias generadas no constituyen asesoría jurídica.
""")
    | llm
)

@tool
def generation_tool(tipo_clausula: str, condiciones: str) -> str:
    """Genera o sugiere cláusulas para contratos de arrendamiento."""
    query = f"{tipo_clausula}: {condiciones}"
    result = chain_generation.invoke(query)
    return result.content

#### Tools

In [35]:
tools = [chat_analysis, filter_search, generation_tool]

### Router prompt

In [36]:
tem_PagPrincipal = """
Eres un asistente especializado en contratos de arrendamiento. 
Tienes acceso a estas herramientas: 
    chat_analysis:      para consultas, resúmenes y análisis generales; 
    filter_search:      cuando el usuario quiere buscar contratos por criterios específicos como fecha, tipo, partes o ubicación; 
    generation_tool:    cuando el usuario quiere crear o sugerir cláusulas, generar borradores o redactar términos. 
Determina cuál usar según la solicitud y extrae los parámetros relevantes.
"""

prompt_agente = ChatPromptTemplate.from_messages([
    ("system", tem_PagPrincipal),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, tools, prompt_agente)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True
                         ,return_intermediate_steps=True) # para ver qué tools usó

NameError: name 'create_tool_calling_agent' is not defined

## Manejo de entradas y salidas

In [ ]:
# Manejas el historial tú mismo (simple y transparente)
chat_history = []

def chat(user_input: str):
    response = executor.invoke({
        "input": user_input,
        "chat_history": chat_history
    })
    
    # actualizar historial
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response["output"]))
    
    # la respuesta tiene:
    # response["output"]                → texto final al usuario
    # response["intermediate_steps"]    → lista de (tool_llamada, resultado)
    
    return response

# Pruebas

In [ ]:
# Usar
result = chat("Hablame sobre los contratos que tienes")

print(result["output"])  # respuesta final

# Para debug, ver qué tool usó:
for action, observation in result["intermediate_steps"]:
    print(f"Tool usada: {action.tool}")
    print(f"Parámetros: {action.tool_input}")
    print(f"Resultado: {observation}")